# VacationPy
----


In [1]:
! jupyter nbextension enable --py gmaps

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_weather = pd.read_csv("output/cities.csv")
cities_weather.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ribeira grande,38.5167,-28.7000,58.89,85,100,14.56,PT,1613098269
1,hobart,-42.8794,147.3294,73.00,40,40,17.27,AU,1613098025
2,kapaa,22.0752,-159.3190,75.20,57,1,6.91,US,1613098116
3,punta arenas,-53.1500,-70.9167,46.40,71,0,6.91,CL,1613098270
4,jamestown,42.0970,-79.2353,14.00,79,40,5.75,US,1613098270


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)

locations = cities_weather[["Lat", "Long"]].astype(float)
humidity = cities_weather["Humidity"].astype(float)

In [24]:
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
ideal_cities = cities_weather.loc[(cities_weather["Wind Speed"] <= 10) & 
                                  (cities_weather["Cloudiness"] == 0) & 
                                  (cities_weather["Max Temp"] >= 70) & 
                                  (cities_weather["Max Temp"] <= 80)].dropna()
ideal_cities.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,atuona,-9.8000,-139.0333,79.59,74,0,9.91,PF,1613098275
112,mandalay,21.9747,96.0836,71.60,49,0,2.30,MM,1613098291
171,mecca,21.4267,39.8261,71.42,65,0,0.56,SA,1613098153
174,veraval,20.9000,70.3667,71.01,39,0,9.10,IN,1613098187
238,pasni,25.2631,63.4710,70.74,48,0,4.76,PK,1613098315


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_df = ideal_cities
hotel_df["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [41]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Long"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"

    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
 
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    print("End of Search")

Retrieving Results for Index 30: atuona.
Closest hotel in atuona is Hôtel Hiva Oa Hanakee Lodge.
------------
End of Search
Retrieving Results for Index 112: mandalay.
Closest hotel in mandalay is Pullman Mandalay Mingalar.
------------
End of Search
Retrieving Results for Index 171: mecca.
Closest hotel in mecca is Pullman Zamzam Makkah.
------------
End of Search
Retrieving Results for Index 174: veraval.
Closest hotel in veraval is HOTEL PLATINUM.
------------
End of Search
Retrieving Results for Index 238: pasni.
Closest hotel in pasni is Juddi Hotel.
------------
End of Search
Retrieving Results for Index 263: les cayes.
Closest hotel in les cayes is Regi Hotel & Bar.
------------
End of Search
Retrieving Results for Index 290: korhogo.
Closest hotel in korhogo is La Rose Blanche Hotel.
------------
End of Search
Retrieving Results for Index 309: vila velha.
Closest hotel in vila velha is Golden Tulip Porto Vitória.
------------
End of Search
Retrieving Results for Index 320: vill

In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [45]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))